# Categorical Data

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

## Background and Motivation

In [2]:
values= pd.Series(['apple', 'orange', 'apple', 'apple']*2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [3]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [4]:
pd.value_counts(values)

C:\Users\Najee\AppData\Local\Temp\ipykernel_15712\3297668723.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(values)


apple     6
orange    2
Name: count, dtype: int64

In [5]:
values = pd.Series([0,1,0,0]*2)
dim = pd.Series(['apple', 'orange'])

In [6]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [7]:
dim

0     apple
1    orange
dtype: object

In [8]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

- This representation as integers is called the categorical or dictionary-encoded representation.
- The array of distinct values can be called the categories, dictonary, or levels of the data.
- The integer values that reference the categories are called the category codes or simply codes.

- Categorical representation can yield significant performance improvements.
- Transformation on categories can be performed without touching the codes.
- Examples:
    - Renaming Categories
    - Appending a new cayegory without changing the order or position of the existing categories

## Categorical Extension Type in Pandas

- pandas has a special `Categorical` extension type of holding data that uses the integer-based categorical represetaton or encoding.
- This is a popular data compression technique.
    - significant faster performance
    - lower memory use, especiall for string data

In [9]:
fruits = ['apple', 'orange', 'apple', 'apple']*2

In [10]:
N = len(fruits)

In [11]:
rng = np.random.default_rng(seed=12345)

In [14]:
df = pd.DataFrame({'fruit': fruits,
                    'basket_id': np.arange(N),
                    'count': rng.integers(3, 15, size=N),
                    'weight': rng.uniform(0, 4, size=N)}, 
                    columns=['basket_id', 'fruit', 'count', 'weight'])

In [15]:
df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


In [ ]:
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object

In [17]:
fruit_cat = df['fruit'].astype('category')

In [18]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

- The values for `fruit_cat` are now an instance of `pandas.Categorical`, which we can access via the `.array` attribute.

In [19]:
c = fruit_cat.array

In [20]:
type(c)

pandas.core.arrays.categorical.Categorical

In [21]:
c

['apple', 'orange', 'apple', 'apple', 'apple', 'orange', 'apple', 'apple']
Categories (2, object): ['apple', 'orange']

In [22]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [23]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

> A useful trick to get a mapping between codes and categories

In [24]:
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

In [26]:
for cp in enumerate(c.categories):
    print(f"type of cp: {type(cp)}. and cp = {cp}")

type of cp: <class 'tuple'>. and cp = (0, 'apple')
type of cp: <class 'tuple'>. and cp = (1, 'orange')


In [31]:
df['fruit'] = df['fruit'].astype('category')
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [32]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [33]:
categories = ['foo', 'bar', 'baz']
codes = [0,1,2,0,0,1]
my_cats_2 = pd.Categorical.from_codes(codes, categories=categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [35]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [36]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']